# Get PA Nursing Home Data

This notebook scrapes data from the Pennsylvania Department of Health, Nursing Care Facility Information database.

The goal is to create a database of nursing homes in Montgomery County, PA that accept Medicaid payments.

## Import dependencies

In [1]:
import re
import requests

from bs4 import BeautifulSoup
import pandas as pd

## Get the data

In [2]:
url = 'https://sais.health.pa.gov/commonpoc/content/publicweb/nhinformation2.asp?COUNTY=Montgomery'
html_page = requests.get(url)
soup = BeautifulSoup(html_page.content, 'html.parser')

In [3]:
# Use to inspect organized/indented HTML
#print(soup.prettify())

### Note
Looking at the HTML revealed that there are table rows (tr) within tr, within tr...   
So, when I loop through the rows, below, I've got to start with the 3rd tr (that is, tr[2:])

## Extract the HTML table, with the target data
The target data are in the last table (`tables[-1]`) on the page.

In [4]:
tables = soup.find_all('table')

### Get column header info

In [5]:
table_headers = tables[-1].find_all('th')

In [6]:
columns = []
for header in table_headers:
    columns.append(header.getText())

### Make DataFrame

In [7]:
table_rows = tables[-1].find_all('tr')

list_of_rows = []

# Remember the tr within tr within tr... need to start at [2:]
for tr in table_rows[2:]:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    
    list_of_rows.append(row)

df = pd.DataFrame(data=list_of_rows, columns=columns)

## Select just the nursing homes that have Medicaid beds
(And drop the 'Select' column, too.)

In [8]:
df_Medicaid = df[df['Payment Options'].str.contains('Medicaid')].drop(columns='Select')

## Clean the DataFrame

### Strip leading and trailing spaces

In [9]:
df_Medicaid['Name/Address/Phone'] = df_Medicaid['Name/Address/Phone'].str.strip()

### Remove all text (.*) after "(Open)"

In [10]:
df_Medicaid['Name/Address/Phone'] = df_Medicaid['Name/Address/Phone'].str.replace('\(Open\).*$', '', regex=True)

In [11]:
df_Medicaid['Name/Address/Phone'][2]

'(BrightenaataAmbler)AMBLER EXTENDED CARE CENTER32 SOUTH BETHLEHEM PIKEAMBLER\xa0PA\xa019002(215)646-7050'

### Remove leading parentheticals

In [12]:
test_data = ['(foo)(bar)ABC(baz)',
             '(bar)ABC(baz)',
             'ABC(baz)',
             'ABC(baz)XYZ']

test_df = pd.DataFrame(data=test_data, columns=['x'])
test_df

,x
0,(foo)(bar)ABC(baz)
1,(bar)ABC(baz)
2,ABC(baz)
3,ABC(baz)XYZ


In [23]:
# Regular expression with two groups:
# 1) One (or more) parentheticals at the beginning of the string, with a lookahead assertion that 
#    the closing paren is immediately followed by a capital letter
# 2) The rest of the line from the first capital letter forward.
parenthetical_regex = '(^\(.*\)+(?=[A-Z]))?([A-Z].*$)'

In [24]:
test_df['x'].str.extract(parenthetical_regex)

,0,1
0,(foo)(bar),ABC(baz)
1,(bar),ABC(baz)
2,NaN,ABC(baz)
3,NaN,ABC(baz)XYZ


In [15]:
df_Medicaid['Name/Address/Phone'] = df_Medicaid['Name/Address/Phone'].str.extract(parenthetical_regex)[1] # Want the second group
df_Medicaid.head()

,Name/Address/Phone,Type of Ownership,Licensure Status,Last Inspection,Size of Facility,Number of Beds,Payment Options,Nursing Hours Per Resident Per Day
0,ABRAMSON RESIDENCE1425 HORSHAM ROADNORTH WALES...,NON-PROFIT,REGULAR,1/27/2020,Large,324,Private PaymentMedicareMedicaid,3.5
2,AMBLER EXTENDED CARE CENTER32 SOUTH BETHLEHEM ...,PROFIT,REGULAR,6/12/2020,Small,100,Private PaymentMedicareMedicaid,3.01
3,ARISTACARE AT MEADOW SPRINGS845 GERMANTOWN PIK...,PROFIT,REGULAR,11/6/2019,Medium,153,Private PaymentMedicareMedicaid,4.84
4,ARTMAN LUTHERAN HOME250 NORTH BETHLEHEM PIKEAM...,NON-PROFIT,REGULAR,1/2/2020,Small,61,Private PaymentMedicareMedicaid,4.67
6,BROOKSIDE HEALTHCARE AND REHABILITATION CENTER...,PROFIT,REGULAR,5/26/2020,Medium,120,Private PaymentMedicareMedicaid,3.24


### Remove phone numbers from Name/Address/Phone and put in own column

In [16]:
# Regex pattern for a phone number in these formats: (XXX) XXX-XXXX, (XXX)XXX-XXXX
# Source: https://regexlib.com/Search.aspx?k=phone
phone_regex = r'(\(\d{3}\) ?\d{3}-\d{4})'

test_str = "This string contains a phone number: (215) 483-7799, let's see if I can pick it out!"

foo = re.search(phone_regex, test_str).group(0) # group(0) returns the entire match string
foo

'(215) 483-7799'

In [17]:
df_Medicaid.insert(1, 'Phone Number', df_Medicaid['Name/Address/Phone'].str.extract(phone_regex))
df_Medicaid['Phone Number'] = df_Medicaid['Phone Number'].str.replace(')', ') ', regex=False)
df_Medicaid.head()

,Name/Address/Phone,Phone Number,Type of Ownership,Licensure Status,Last Inspection,Size of Facility,Number of Beds,Payment Options,Nursing Hours Per Resident Per Day
0,ABRAMSON RESIDENCE1425 HORSHAM ROADNORTH WALES...,(215) 371-3000,NON-PROFIT,REGULAR,1/27/2020,Large,324,Private PaymentMedicareMedicaid,3.5
2,AMBLER EXTENDED CARE CENTER32 SOUTH BETHLEHEM ...,(215) 646-7050,PROFIT,REGULAR,6/12/2020,Small,100,Private PaymentMedicareMedicaid,3.01
3,ARISTACARE AT MEADOW SPRINGS845 GERMANTOWN PIK...,(610) 279-7300,PROFIT,REGULAR,11/6/2019,Medium,153,Private PaymentMedicareMedicaid,4.84
4,ARTMAN LUTHERAN HOME250 NORTH BETHLEHEM PIKEAM...,(215) 643-6335,NON-PROFIT,REGULAR,1/2/2020,Small,61,Private PaymentMedicareMedicaid,4.67
6,BROOKSIDE HEALTHCARE AND REHABILITATION CENTER...,(215) 884-6776,PROFIT,REGULAR,5/26/2020,Medium,120,Private PaymentMedicareMedicaid,3.24


### Remove phone number from Name/Address/Phone and rename to just Name/Address

In [18]:
df_Medicaid['Name/Address/Phone'] = df_Medicaid['Name/Address/Phone'].str.replace(phone_regex, '')
df_Medicaid.rename(columns={'Name/Address/Phone' : 'Name/Address'}, inplace=True)
df_Medicaid['Name/Address'][0]

'ABRAMSON RESIDENCE1425 HORSHAM ROADNORTH WALES\xa0PA\xa019454'

### Split Name and Address of the facility

In [19]:
name_regex = r'^(.+?)\d'
df_Medicaid.insert(0, 'Name', df_Medicaid['Name/Address'].str.extract(name_regex))
df_Medicaid['Name'].head()

0                                ABRAMSON RESIDENCE
2                       AMBLER EXTENDED CARE CENTER
3                      ARISTACARE AT MEADOW SPRINGS
4                              ARTMAN LUTHERAN HOME
6    BROOKSIDE HEALTHCARE AND REHABILITATION CENTER
Name: Name, dtype: object

In [20]:
address_regex = r'(\d.*)$'
df_Medicaid.insert(1, 'Address', df_Medicaid['Name/Address'].str.extract(address_regex))
df_Medicaid['Address'].head()

0           1425 HORSHAM ROADNORTH WALES PA 19454
2          32 SOUTH BETHLEHEM PIKEAMBLER PA 19002
3    845 GERMANTOWN PIKEPLYMOUTH MEETING PA 19462
4         250 NORTH BETHLEHEM PIKEAMBLER PA 19002
6               2630 WOODLAND ROADROSLYN PA 19001
Name: Address, dtype: object

In [21]:
# Drop 'Name/Address'
df_Medicaid.drop(columns=['Name/Address'], inplace=True)

## Write out the data to a CSV file

In [22]:
df_Medicaid.to_csv('output/Montgomery_County_PA_nursing_homes_with_Medicaid_beds.csv', index=False)